In [ ]:
# This script is for creating road patches from 20p data that correspond to road patches of 5p data. 
# Paula's road vectors are used for this purpose. Some of these road vectors may contain NaN values.

In [1]:
east_min = 395000.0 # 395500.0 
east_max = 407000.0 # 400500.0 
north_min = 6783000.0 # 6783500.0 
north_max = 6795000.0 # 6788500.0 

print(east_max-east_min)
print(north_max-north_min)

12000.0
12000.0


In [4]:
import os
import re
import math
import shutil
import laspy
import numpy as np

def parse_tile_coordinates(filename):
    """Extract x and y tile coordinates from the filename."""
    match = re.search(r"tile_x_(\d+)_y_(\d+)\.laz", filename)
    if match:
        x = int(match.group(1))
        y = int(match.group(2))
        return x, y
    return None

def find_n_closest_tiles(point, folder_path, n):
    """Find the four tiles whose (x, y) are closest to the given 3D point's (x, y)."""
    point_x, point_y, _ = point  # Ignore z
    distances = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.laz'):
            coords = parse_tile_coordinates(filename)
            if coords:
                tile_x, tile_y = coords

                dist = math.hypot(tile_x - point_x, tile_y - point_y)
                distances.append((dist, filename))

    # Sort by distance and get the four closest files
    distances.sort()
    closest_files = [filename for _, filename in distances[:n]]

    return closest_files

In [ ]:
folder_road_vectors = r"C:\Users\telukkari\Documents\Python\time_series_analysis\data\road_vecs_las_Padasjoki_20231023"
folder_tiles = r"E:\ALS_DATA\NLS_20p_EVO_TILES" 
output_folder = r"E:\ALS_DATA\NLS_20p_EVO_road_patches"

for file_road_vector in os.listdir(folder_road_vectors):
    file_path_road_vector = os.path.join(folder_road_vectors, file_road_vector)
    road_vector = laspy.read(file_path_road_vector)
    points_road_vector = np.vstack((road_vector.x, road_vector.y, road_vector.z)).T

    east_min = 395000.0 # 395500.0 
    east_max = 407000.0 # 400500.0 
    north_min = 6783000.0 # 6783500.0 
    north_max = 6795000.0 # 6788500.0 

    if points_road_vector.shape[0] == 0:
        continue
    if points_road_vector[0,0] > east_min and points_road_vector[0,0] < east_max and points_road_vector[0,1] > north_min and points_road_vector[0,1] < north_max:
        print(f"Loaded {len(points_road_vector)} trajectory points from {file_path_road_vector}")

        # TODO check that road vector does not contain NaN values
        # TODO add points if the distance between two consecutive points is too large

        all_nearby_points = None

        for point in points_road_vector:
            #print(f"Trajectory Point: {point}")
            closest_tiles = find_n_closest_tiles(point, folder_tiles, n=4)

            for tile_name in closest_tiles:
                tile_path = os.path.join(folder_tiles, tile_name)
                tile = laspy.read(tile_path)  

                # tile_points = np.vstack((tile.x, tile.y, tile.z, tile.intensity)).T #, tile.scan_angle_rank, tile.return_number, tile.gps_time, tile.Amplitude, tile.Deviation)).T

                # Compute horizontal distance to current trajectory point
                dx = tile.x - point[0]
                dy = tile.y - point[1]
                horizontal_dist = np.sqrt(dx**2 + dy**2)

                nearby_mask = horizontal_dist <= 10
                nearby_points = tile.points[nearby_mask]
                # nearby_points = tile_points[nearby_mask]
                if nearby_points.size > 0:
                    if all_nearby_points == None:
                        all_nearby_points = nearby_points
                    else:
                        all_nearby_points = np.hstack([all_nearby_points, nearby_points])
                    # all_road_patch_points.append(nearby_points)

        print(f"Found {len(all_nearby_points)} nearby points.")

        # Stack all filtered points into one numpy array
        # if all_nearby_points:
        #     stacked_points = np.vstack(all_road_patch_points)
        # else:
        #     stacked_points = np.empty((0, 9))

        if all_nearby_points == None:
            all_nearby_points = np.empty((0, 9))

        # 1. Remove duplicate rows (points)
        unique_points = np.unique(all_nearby_points, axis=0)

        # 2. Create a new LAS file with the unique points
        header = laspy.LasHeader(point_format=6, version="1.4")
        las = laspy.LasData(header)

        las.points = unique_points

        # las.x = unique_points[:, 0]
        # las.y = unique_points[:, 1]
        # las.z = unique_points[:, 2]
        # las.intensity = unique_points[:,3]
        # las.scan_angle = unique_points[:,4]
        # las.return_number = unique_points[:,5]
        # las.gps_time = unique_points[:,6]
        # las.Amplitude = unique_points[:,7]
        # las.Deviation = unique_points[:,8]

        # 3. Save the LAS file
        output_path = os.path.join(output_folder, f"{file_road_vector}")
        # las.write(output_path)

        print(f"Saved {len(unique_points)} unique points to {output_path}")


Loaded 7 trajectory points from C:\Users\telukkari\Documents\Python\time_series_analysis\data\road_vecs_las_Padasjoki_20231023\1077366617_45474.las
Found 13 nearby points in tile ['tile_x_396125_y_6783725.laz', 'tile_x_396075_y_6783725.laz', 'tile_x_396125_y_6783775.laz', 'tile_x_396075_y_6783775.laz']
Saved 31255 unique points to E:\ALS_DATA\NLS_20p_EVO_road_patches\1077366617_45474.las
Loaded 38 trajectory points from C:\Users\telukkari\Documents\Python\time_series_analysis\data\road_vecs_las_Padasjoki_20231023\1080419324_45480.las
Found 74 nearby points in tile ['tile_x_397775_y_6783625.laz', 'tile_x_397775_y_6783675.laz', 'tile_x_397725_y_6783625.laz', 'tile_x_397725_y_6783675.laz']
Saved 232858 unique points to E:\ALS_DATA\NLS_20p_EVO_road_patches\1080419324_45480.las
Loaded 154 trajectory points from C:\Users\telukkari\Documents\Python\time_series_analysis\data\road_vecs_las_Padasjoki_20231023\1080419383_45429.las
Found 303 nearby points in tile ['tile_x_396475_y_6785325.laz', 't